# 중복청약 IPO 캘린더 ver2

### version history
ver1)
 - date : 2021년
 - description : 
     공모청약일정 게시판(http://www.38.co.kr/html/fund/index.htm?o=k&page={0})에 노출되는 테이블에서 주간사 내역 추출
 
ver2)
 - date : 2022-12-14
 - description :
     공모청약일정 게시판(http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2022&str2=all)에 노출되는 테이블에서 각 청약별 상세 페이지 접속 후에 주간사 내역 테이블(<table class="view_tb"......>) 파싱으로 변경


### 1. 조회기간 입력

In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup as bs

#from konlpy.utils import pprint

import re

import time
from datetime import date
from dateutil.relativedelta import relativedelta

from pprintpp import pprint

In [2]:
# 올해 1월 1일을 시작일자
# 현재 처리시점의 익월 말일을 종료일자로 입력해주세요
print("검색 시작일자를 입력해주세요. (YYYY-MM-DD) >>")
inq_st_dt = input()

print("검색 종료일자를 입력해주세요. (YYYY-MM-DD) >>")
inq_nd_dt = input()

검색 시작일자를 입력해주세요. (YYYY-MM-DD) >>
2024-01-01
검색 종료일자를 입력해주세요. (YYYY-MM-DD) >>
2024-12-31


### 2. ipostock 공모청약일정 게시판 청약별 링크 수집

In [3]:
itr_flag=True

ipo_list=list()

pge_cnt=1

while True == itr_flag :
    url_list = "http://www.ipostock.co.kr/sub03/ipo04.asp?str1={0}&str2=all&str3=&str4=&page={1}"
    
    # ~ 2022.11
    #    http://www.38.co.kr/html/fund/index.htm?o=k&page={0}
    # 2022.12 ~
    #    http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2022&str2=all
    #    http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2022&str2=all&str3=&str4=&page=1
    #    http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2022&str2=all&str3=&str4=&page=2
    #    http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2022&str2=all&str3=&str4=&page=7
    
    url = url_list.format(inq_st_dt[:4], pge_cnt)
    print(url)
    response = requests.get(url)
    response.encoding='utf-8'
    
    soup=bs(response.text, 'html.parser')
    a_tag_list=soup.select('a')

    tmp_ipo_list=list()

    for a_tag in a_tag_list :
        
        if 'href' in a_tag.attrs :
            if -1 != a_tag.attrs['href'].find("/view_pg/view_01.asp?code=") :
                tmp_ipo_list.append(a_tag.attrs['href'])
            elif -1 != a_tag.attrs['href'].find("/view_pg/view_04.asp?code=") :
                tmp_ipo_list.append(a_tag.attrs['href'])
    
    if 0 < len(tmp_ipo_list) :
        ipo_list.append(tmp_ipo_list)
    else :
        itr_flag = False
    
    pge_cnt=pge_cnt+1
    
    time.sleep(2) # sleep 2 sec
    
ipo_list[:10]

http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2024&str2=all&str3=&str4=&page=1
http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2024&str2=all&str3=&str4=&page=2
http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2024&str2=all&str3=&str4=&page=3
http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2024&str2=all&str3=&str4=&page=4
http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2024&str2=all&str3=&str4=&page=5
http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2024&str2=all&str3=&str4=&page=6
http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2024&str2=all&str3=&str4=&page=7
http://www.ipostock.co.kr/sub03/ipo04.asp?str1=2024&str2=all&str3=&str4=&page=8


[['/view_pg/view_04.asp?code=B202111151&schk=2',
  '/view_pg/view_04.asp?code=B202406102&schk=2',
  '/view_pg/view_04.asp?code=B202407051&schk=2',
  '/view_pg/view_04.asp?code=B201906213&schk=2',
  '/view_pg/view_04.asp?code=B202411141&schk=2',
  '/view_pg/view_04.asp?code=B202405091&schk=2',
  '/view_pg/view_04.asp?code=B202405131&schk=2',
  '/view_pg/view_04.asp?code=B202309261&schk=2',
  '/view_pg/view_04.asp?code=B202408262&schk=2',
  '/view_pg/view_04.asp?code=B202405302&schk=2',
  '/view_pg/view_04.asp?code=B202408261&schk=2',
  '/view_pg/view_04.asp?code=B202409271&schk=2',
  '/view_pg/view_04.asp?code=B202408202&schk=2',
  '/view_pg/view_01.asp?code=B202408011&schk=2',
  '/view_pg/view_01.asp?code=B202405163&schk=2',
  '/view_pg/view_04.asp?code=B202405202&schk=2',
  '/view_pg/view_04.asp?code=B202408082&schk=2',
  '/view_pg/view_04.asp?code=B202312112&schk=2',
  '/view_pg/view_04.asp?code=B202404183&schk=2',
  '/view_pg/view_04.asp?code=B202405164&schk=2'],
 ['/view_pg/view_04

### 3. 종목별 상세정보 스크래핑

In [4]:
ipo_df_f = pd.DataFrame(columns=['url', '종목명', '시장', '청약시작일자', '청약종료일자', '주간사'])
ipo_df_f.set_index("url", inplace = True)

for itr_ipo_list in ipo_list :
    
    for itr_ipo in itr_ipo_list :
    
        itr_ipo_url='http://www.ipostock.co.kr' + itr_ipo
    
        itr_ipo_url=itr_ipo_url.replace('schk=2','gmenu=')
    
        print( 'url : ' + itr_ipo_url )
        
        #http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202208181&schk=2
        #http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202211161&gmenu=
        #http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202210113&gmenu=
        
        response = requests.get(itr_ipo_url)
        response.encoding='utf-8'
        
        soup=bs(response.text, 'html.parser')
        #print(response.text)
        
        ############################ 청약종목명 ############################
        
        ipo_stock_nm = soup.find('strong', 'view_tit').text # 청약종목명
        print('청약종목명 : ' + ipo_stock_nm)
        
        ####################################################################
    
    
        ############################ 공모철회 여부 검증 ############################
        
        is_revocation=False
        
        #pprint(soup.select('td > font'))
        
        for font_tag in soup.select('td > font') :
            
            #pprint(font_tag.text)
            
            if -1 != font_tag.text.find('철회') :
                is_revocation=True
                print('공모철회...')
                break
        
        if True == is_revocation :
            print("\n\n\n")
            continue
        
        ############################################################################
    
    
        ######################### 증권시장 정보 추출 #########################
        
        ipo_market = '코넥스'
        
        img_tag_list = soup.find_all('img')
        
        for img_tag in img_tag_list :
            if 'src' in img_tag.attrs :
                if "/image/contents/co.gif" == img_tag.attrs['src'] :
                    ipo_market = '코스닥'
                elif "/image/contents/u.gif" == img_tag.attrs['src'] :
                    ipo_market = '유가증권시장'
        
        print('증권시장 : ' + ipo_market)
        
        ######################################################################
    
    
        ############################ 공모청약 일정 ############################
        
        ipo_schedule=pd.read_html(
            str(soup.find_all('table', "view_tb")[0])
        )[0] # 공모청약일정
        
        ipo_schedule=ipo_schedule.T
        ipo_schedule_first_row = ipo_schedule.iloc[0]
        ipo_schedule = ipo_schedule[1:]
        ipo_schedule.columns = ipo_schedule_first_row
        
        #print( ipo_schedule )
        
        ipo_start_date=list(ipo_schedule['공모청약일'].str.split('~').str[0])[0].strip()
        ipo_end_date=list(ipo_schedule['공모청약일'].str.split('~').str[1])[0].strip()

        ipo_start_date=ipo_start_date.replace('.','-')
        ipo_end_date=ipo_end_date.replace('.','-')
        
        date_pattern1 = re.compile('[1-9][0-9][0-9][0-9]-[0-9]?[0-9]-[0-9]?[0-9]')
        match_flag1 = date_pattern1.search(ipo_end_date)
        
        date_pattern2 = re.compile('[0-9]?[0-9]-[0-9]?[0-9]')
        match_flag2 = date_pattern2.search(ipo_end_date)
        
        #print(match_flag1)
        #print(match_flag2)
        
        if match_flag2 :
            ipo_end_date=ipo_start_date[:4] + '-' + ipo_end_date
            
            ipo_start_date_format=date.fromisoformat(ipo_start_date)
            ipo_end_date_format=date.fromisoformat(ipo_end_date)
            
            if ipo_start_date_format > ipo_end_date_format :
                ipo_end_date_format=ipo_end_date_format + relativedelta(years=1)
                ipo_end_date=ipo_end_date_format.strftime("%Y-%m-%d")
        
        print('청약시작일자 : ' + ipo_start_date)
        print('청약종료일자 : ' + ipo_end_date)
        
        #######################################################################
        
        ############################ 주간사/인수사 목록 추출 ############################
        
        ipo_brokers=pd.read_html(
            str(soup.find_all('table', "view_tb")[3]), header=0
        )[0] # 일반청약 주간사
        
        ipo_brokers['증권회사']=ipo_brokers['증권회사'].str.replace("\(주[)]?", "", regex=True)
        
        print( '주간사 : ' + ','.join(list(ipo_brokers['증권회사'])) )
        
        #################################################################################
        
        ipo_df_f=ipo_df_f.append(
            pd.DataFrame(
                data={
                    'url' : [itr_ipo_url]
                    , '종목명' : [ipo_stock_nm]
                    , '시장' : [ipo_market]
                    , '청약시작일자' : [ipo_start_date]
                    , '청약종료일자' : [ipo_end_date]
                    , '주간사' : [','.join(list(ipo_brokers['증권회사']))]
                }
            ).set_index('url')
        )
        
        #pprint(ipo_df_f)
        
        print("\n\n\n")
        
        time.sleep(2) # sleep 2 sec


print("\n\n===============================================")    

ipo_df_f

url : http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202111151&gmenu=
청약종목명 : 파인메딕스
증권시장 : 코스닥
청약시작일자 : 2024-12-16
청약종료일자 : 2024-12-17
주간사 : 한국투자증권




url : http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202406102&gmenu=
청약종목명 : 쓰리에이로직스
증권시장 : 코스닥
청약시작일자 : 2024-12-13
청약종료일자 : 2024-12-16
주간사 : 미래에셋증권,신한투자증권




url : http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202407051&gmenu=
청약종목명 : 신한제14호스팩
증권시장 : 코스닥
청약시작일자 : 2024-12-11
청약종료일자 : 2024-12-12
주간사 : 신한투자증권




url : http://www.ipostock.co.kr/view_pg/view_04.asp?code=B201906213&gmenu=
청약종목명 : 듀켐바이오
증권시장 : 코스닥
청약시작일자 : 2024-12-11
청약종료일자 : 2024-12-12
주간사 : NH투자증권




url : http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202411141&gmenu=
청약종목명 : 키움제10호스팩
증권시장 : 코스닥
청약시작일자 : 2024-12-11
청약종료일자 : 2024-12-12
주간사 : 키움증권




url : http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202405091&gmenu=
청약종목명 : 아이에스티이
증권시장 : 코스닥
청약시작일자 : 2024-12-10
청약종료일자 : 2024-12-11
주간사 : KB증권




url : http://www.ipostock.co.kr/view_pg/view_0

,종목명,시장,청약시작일자,청약종료일자,주간사
url,,,,,
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202111151&gmenu=,파인메딕스,코스닥,2024-12-16,2024-12-17,한국투자증권
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202406102&gmenu=,쓰리에이로직스,코스닥,2024-12-13,2024-12-16,"미래에셋증권,신한투자증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202407051&gmenu=,신한제14호스팩,코스닥,2024-12-11,2024-12-12,신한투자증권
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B201906213&gmenu=,듀켐바이오,코스닥,2024-12-11,2024-12-12,NH투자증권
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202411141&gmenu=,키움제10호스팩,코스닥,2024-12-11,2024-12-12,키움증권
...,...,...,...,...,...
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202307105&gmenu=,포스뱅크,코스닥,2024-01-17,2024-01-18,하나증권
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202308171&gmenu=,현대힘스,코스닥,2024-01-17,2024-01-18,미래에셋증권
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202309071&gmenu=,에이치비인베스트먼트,코스닥,2024-01-16,2024-01-17,NH투자증권


### 4. 인수회사 2개 이상인 종목만 필터링

In [5]:
ipo_df_f=ipo_df_f.loc[True == ipo_df_f['주간사'].str.contains(',')]#인수회사가 2개 이상인 경우만 취합
ipo_df_f

,종목명,시장,청약시작일자,청약종료일자,주간사
url,,,,,
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202406102&gmenu=,쓰리에이로직스,코스닥,2024-12-13,2024-12-16,"미래에셋증권,신한투자증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202408262&gmenu=,엠앤씨솔루션,유가증권시장,2024-12-05,2024-12-06,"KB증권,삼성증권,키움증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202405292&gmenu=,더본코리아,유가증권시장,2024-10-28,2024-10-29,"한국투자증권,NH투자증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202404181&gmenu=,클로봇,코스닥,2024-10-16,2024-10-17,"미래에셋증권,신영증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202407292&gmenu=,씨메스,코스닥,2024-10-15,2024-10-16,"삼성증권,유진투자증권,유안타증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202404301&gmenu=,한켐,코스닥,2024-10-07,2024-10-08,"신영증권,유진투자증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202404221&gmenu=,제닉스,코스닥,2024-09-19,2024-09-20,"신영증권,KB증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202404081&gmenu=,엠83,코스닥,2024-08-12,2024-08-13,"신영증권,유진투자증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202404123&gmenu=,산일전기,유가증권시장,2024-07-18,2024-07-19,"미래에셋증권,삼성증권"


### 5. 조회기간 필터 적용

In [6]:
ipo_df_f=ipo_df_f.loc[
    #(
    #    ( inq_st_dt<=ipo_df_f['청약시작일자'] ) 
    #    &
    #    ( ipo_df_f['청약시작일자']<=inq_nd_dt ) 
    #)
    #|
    (
        ( inq_st_dt<=ipo_df_f['청약종료일자'] ) 
        &
        ( ipo_df_f['청약종료일자']<=inq_nd_dt ) 
    )
]
ipo_df_f

,종목명,시장,청약시작일자,청약종료일자,주간사
url,,,,,
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202406102&gmenu=,쓰리에이로직스,코스닥,2024-12-13,2024-12-16,"미래에셋증권,신한투자증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202408262&gmenu=,엠앤씨솔루션,유가증권시장,2024-12-05,2024-12-06,"KB증권,삼성증권,키움증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202405292&gmenu=,더본코리아,유가증권시장,2024-10-28,2024-10-29,"한국투자증권,NH투자증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202404181&gmenu=,클로봇,코스닥,2024-10-16,2024-10-17,"미래에셋증권,신영증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202407292&gmenu=,씨메스,코스닥,2024-10-15,2024-10-16,"삼성증권,유진투자증권,유안타증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202404301&gmenu=,한켐,코스닥,2024-10-07,2024-10-08,"신영증권,유진투자증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202404221&gmenu=,제닉스,코스닥,2024-09-19,2024-09-20,"신영증권,KB증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202404081&gmenu=,엠83,코스닥,2024-08-12,2024-08-13,"신영증권,유진투자증권"
http://www.ipostock.co.kr/view_pg/view_04.asp?code=B202404123&gmenu=,산일전기,유가증권시장,2024-07-18,2024-07-19,"미래에셋증권,삼성증권"


### 6. 중복청약확인 대상 종목정보 엑셀파일로 저장

In [7]:
ipo_df_f.to_excel(
    "./output/ipo_schedule_" + inq_st_dt + "_" + inq_nd_dt + ".xlsx" # directory and file name to write
    , sheet_name = 'Sheet1'
    , na_rep = 'NaN'
    #, float_format = "%.2f"
    , header = True
    #, columns = ["group", "value_1", "value_2"] # if header is False
    , index = False
    #, index_label = "id", # if index is True
    , startrow = 1
    , startcol = 1
    #, engine = 'xlsxwriter'
    , freeze_panes = (2, 0)
) 

### 7. 중복청약확인 대상 IPO 캘린더 생성

In [8]:
from calendarUtils import *

now = datetime.datetime.now() 
print(
    now.year
    , now.month
    , now.day
    , now.hour
    , now.minute
    , now.second
) 
year = now.year

2024 12 9 5 33 21


In [9]:

#create workbook 
wb = Workbook()

for month in range(1,13) : 
    if month == 1 : 
        ws = wb.active 
        ws.title = "1월" 
    else : 
        ws = wb.create_sheet(str(month) + "월") 
    
    set_title(ws, str(month) + "월") 
    set_weeks_title(ws) 
    set_ipo_calendar(ws, year, month, ipo_df_f) 
    
wb.save(filename="./output/IPO일정_" + str(year) +".xlsx")

2024-02-
15
- 에이피알
  * 신한투자증권
  * 하나증권
2024-02-
27
- 케이엔알시스템
  * DB금융투자
  * NH투자증권
2024-04-
26
- 에이치디현대마린솔루션
  * KB증권
  * 신한투자증권
  * 하나증권
  * 대신증권
  * 삼성증권
2024-06-
14
- 신한글로벌액티브리츠
  * 신한투자증권
  * 한국투자증권
2024-06-
17
- 에스오에스랩
  * 한국투자증권
  * 비엔케이투자
2024-06-
21
- 이노스페이스
  * 미래에셋증권
  * 신한투자증권
2024-07-
3
- 시프트업
  * 한국투자증권
  * NH투자증권
  * 신한투자증권
2024-07-
19
- 산일전기
  * 미래에셋증권
  * 삼성증권
2024-08-
13
- 엠83
  * 신영증권
  * 유진투자증권
2024-09-
20
- 제닉스
  * 신영증권
  * KB증권
2024-10-
8
- 한켐
  * 신영증권
  * 유진투자증권
2024-10-
16
- 씨메스
  * 삼성증권
  * 유진투자증권
  * 유안타증권
2024-10-
17
- 클로봇
  * 미래에셋증권
  * 신영증권
2024-10-
29
- 더본코리아
  * 한국투자증권
  * NH투자증권
2024-12-
6
- 엠앤씨솔루션
  * KB증권
  * 삼성증권
  * 키움증권
2024-12-
16
- 쓰리에이로직스
  * 미래에셋증권
  * 신한투자증권
